In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

df=pd.read_csv('online_sales_dataset.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount,PaymentMethod,ShippingCost,Category,SalesChannel,ReturnStatus,ShipmentProvider,WarehouseLocation,OrderPriority
0,221958,SKU_1964,White Mug,38,2020-01-01 00:00,1.71,37039.0,Australia,0.470000,Bank Transfer,10.79,Apparel,In-store,Not Returned,UPS,London,Medium
1,771155,SKU_1241,White Mug,18,2020-01-01 01:00,41.25,19144.0,Spain,0.190000,paypall,9.51,Electronics,Online,Not Returned,UPS,Rome,Medium
2,231932,SKU_1501,Headphones,49,2020-01-01 02:00,29.11,50472.0,Germany,0.350000,Bank Transfer,23.03,Electronics,Online,Returned,UPS,Berlin,High
3,465838,SKU_1760,Desk Lamp,14,2020-01-01 03:00,76.68,96586.0,Netherlands,0.140000,paypall,11.08,Accessories,Online,Not Returned,Royal Mail,Rome,Low
4,359178,SKU_1386,USB Cable,-30,2020-01-01 04:00,-68.11,NaN,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium


In [16]:
df.shape

(49782, 17)

In [17]:
df.duplicated().sum()

0

In [18]:
df.isnull().sum()

InvoiceNo               0
StockCode               0
Description             0
Quantity                0
InvoiceDate             0
UnitPrice               0
CustomerID           4978
Country                 0
Discount                0
PaymentMethod           0
ShippingCost         2489
Category                0
SalesChannel            0
ReturnStatus            0
ShipmentProvider        0
WarehouseLocation    3485
OrderPriority           0
dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49782 entries, 0 to 49781
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   InvoiceNo          49782 non-null  int64  
 1   StockCode          49782 non-null  object 
 2   Description        49782 non-null  object 
 3   Quantity           49782 non-null  int64  
 4   InvoiceDate        49782 non-null  object 
 5   UnitPrice          49782 non-null  float64
 6   CustomerID         44804 non-null  float64
 7   Country            49782 non-null  object 
 8   Discount           49782 non-null  float64
 9   PaymentMethod      49782 non-null  object 
 10  ShippingCost       47293 non-null  float64
 11  Category           49782 non-null  object 
 12  SalesChannel       49782 non-null  object 
 13  ReturnStatus       49782 non-null  object 
 14  ShipmentProvider   49782 non-null  object 
 15  WarehouseLocation  46297 non-null  object 
 16  OrderPriority      497

In [20]:
df['ShippingCost']=df['ShippingCost'].fillna(0)

replace the value from null to 0 in ShippingCost column

In [21]:
df.isnull().sum()

InvoiceNo               0
StockCode               0
Description             0
Quantity                0
InvoiceDate             0
UnitPrice               0
CustomerID           4978
Country                 0
Discount                0
PaymentMethod           0
ShippingCost            0
Category                0
SalesChannel            0
ReturnStatus            0
ShipmentProvider        0
WarehouseLocation    3485
OrderPriority           0
dtype: int64

In [29]:
# Create SQLite DB in memory
conn = sqlite3.connect(':memory:')

# Write DataFrame to SQL table
df.to_sql('my_table', conn, index=False, if_exists='replace')

query='''
select  sum(UnitPrice*Quantity) as total_sum from my_table
'''
df_sql=pd.read_sql_query(query,conn)
print(df_sql)

     total_sum
0  60142533.09


that's the total sale 


In [42]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


converted InvoiceDate column from object to date and time 

In [45]:
df.to_sql('my_table', conn, index=False, if_exists='replace')

query = '''
SELECT strftime('%m', InvoiceDate) AS month
FROM my_table
'''

df_sql = pd.read_sql_query(query, conn)
print(df_sql)


      month
0        01
1        01
2        01
3        01
4        01
...     ...
49777    09
49778    09
49779    09
49780    09
49781    09

[49782 rows x 1 columns]


extracted month from date

In [48]:
df.to_sql('my_table', conn, index=False, if_exists='replace')

query = '''
SELECT strftime('%m', InvoiceDate) AS month,CustomerId,Country 
from my_table
group by 
month
'''

df_sql = pd.read_sql_query(query, conn)
print(df_sql)

   month  CustomerID    Country
0     01     37039.0  Australia
1     02     96337.0  Australia
2     03     57652.0     Sweden
3     04     79534.0     Sweden
4     05     44302.0   Portugal
5     06     77056.0      Italy
6     07     54067.0      Italy
7     08     58795.0     France
8     09     65561.0      Italy
9     10         NaN     Sweden
10    11     56780.0      Italy
11    12         NaN     Norway


grouped the customer based on month

In [49]:
query = "SELECT  count(CustomerID ) as count FROM my_table "
result_df = pd.read_sql_query(query, conn)
print(result_df)

   count
0  44804


there are 44804 customer who have ordered

In [50]:
df.to_sql('my_table', conn, index=False, if_exists='replace')

query = '''
SELECT strftime('%m', InvoiceDate) AS month,CustomerId,Country 
from my_table
group by 
month
order by Country
'''

df_sql = pd.read_sql_query(query, conn)
print(df_sql)

   month  CustomerID    Country
0     01     37039.0  Australia
1     02     96337.0  Australia
2     08     58795.0     France
3     06     77056.0      Italy
4     07     54067.0      Italy
5     09     65561.0      Italy
6     11     56780.0      Italy
7     12         NaN     Norway
8     05     44302.0   Portugal
9     03     57652.0     Sweden
10    04     79534.0     Sweden
11    10         NaN     Sweden


used order by aggregate 